In [1]:
import pandas as pd

In [16]:
# Updating parking zone input file for future years (parking_costs.csv)
df = pd.read_csv('R:\e2projects_two\SoundCast\Inputs\db_inputs\parking_costs.csv')

In [13]:
df['year'].unique()
df.head()

,ENS,HR_COST,DAY_COST,year
0,gz1,390,1464,2014
1,gz2,139,614,2014
2,gz3,129,812,2014
3,gz4,223,915,2014
4,gz5,246,1111,2014


In [ ]:
# Develop data for 2035 by interpolating between 2030 and 2040
df_2030 = df[df['year'] == 2030].copy()
df_2040 = df[df['year'] == 2040].copy()
df_2035 = df_2030.merge(df_2040, on='ENS', suffixes=('_2030', '_2040'))
df_2035['HR_COST'] = df_2035['HR_COST_2030'] + (df_2035['HR_COST_2040'] - df_2035['HR_COST_2030']) / 2
df_2035['DAILY_COST'] = df_2035['DAY_COST_2030'] + (df_2035['DAY_COST_2040'] - df_2035['DAY_COST_2030']) / 2
df_2035['year'] = 2035
df_2035 = df_2035[['ENS','HR_COST', 'DAILY_COST', 'year']]
df = pd.concat([df,df_2035], ignore_index=True, sort=False, axis=0).fillna(0)

# Sort values by ensemble number
df['zone_id'] = df['ENS'].astype(str).apply(lambda x: x.split('gz')[-1]).astype('int')
df.sort_values(['year','zone_id'], inplace=True)
df.drop('zone_id', axis=1, inplace=True)
df.to_clipboard()

df.to_csv('R:\e2projects_two\SoundCast\Inputs\db_inputs\parking_costs.csv', index=False)

In [ ]:
# Analysis: compare parcel median costs to the ensemble input data


# Parking costs are available on parcels (via parking survey data, the latest being from 2022)

# We can compare the ensemble (zone) data for 2023 to see if it's reasonably matching the parcel data

# Base year data doesn't need to be udpated because parcel parking data is updated through urbansim
# 2023 was copied from 2018 but these data won't actually be used 
df_parcel = pd.read_csv(r'R:\e2projects_two\SoundCast\Inputs\dev\landuse\2023\23_on_23_v2\parcels_urbansim.txt', sep='\s+')

In [43]:
df_parcel.head()

,aparks,empedu_p,empfoo_p,empgov_p,empind_p,empmed_p,empofc_p,empoth_p,empret_p,emprsc_p,...,ppricdyp,pprichrp,sfunits,sqft_p,stugrd_p,stuhgh_p,stuuni_p,taz_p,xcoord_p,ycoord_p
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,85802.593750,0,0,0,1019,1.292255e+06,162728.617255
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.000000,0,0,0,1018,1.291832e+06,164041.742835
2,0,0,0,0,0,0,2,0,0,0,...,0,0,0,6000.066365,0,0,0,1018,1.291595e+06,164048.669737
3,0,0,0,0,2,0,0,2,2,0,...,0,0,0,7200.084044,0,0,0,1018,1.291540e+06,164050.178628
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,6068.041674,0,0,0,1018,1.291479e+06,164042.397388


In [44]:
# Load parking zone file
df_parking_zones = pd.read_csv(r'R:\e2projects_two\SoundCast\Inputs\db_inputs\parking_zones.csv')
df_parking_costs = pd.read_csv(r'R:\e2projects_two\SoundCast\Inputs\db_inputs\parking_costs.csv')

In [45]:
df_parking = df_parking_costs.merge(df_parking_zones, on='ENS')
df_parking = df_parking[df_parking['year'] == 2018]

In [46]:
# df_parking

In [47]:
df_parcel = df_parcel.merge(df_parking, left_on='taz_p', right_on='TAZ')

In [48]:
# Get median price for paid parking zones in each ensemble zone
df_results = df_parcel[df_parcel['ppricdyp'] > 0].groupby('ENS').median()[['HR_COST','DAY_COST','ppricdyp','pprichrp']]

In [49]:
# df_results[['HR_COST','DAY_COST']] = df_results[['HR_COST','DAY_COST']]/100

In [50]:
df_results

,HR_COST,DAY_COST,ppricdyp,pprichrp
ENS,,,,
gz1,390.0,1464.0,2400.0,400.0
gz10,383.0,1669.0,2650.0,500.0
gz11,415.0,1787.0,2500.0,400.0
gz16,0.0,0.0,1500.0,400.0
gz19,0.0,0.0,800.0,200.0
gz2,139.0,614.0,1200.0,200.0
gz3,129.0,812.0,900.0,200.0
gz4,223.0,915.0,800.0,200.0
gz5,246.0,1111.0,1700.0,400.0


In [51]:
df_results = df_parcel[df_parcel['ppricdyp'] > 0].groupby('ENS').mean()[['HR_COST','DAY_COST','ppricdyp','pprichrp']]

In [52]:
df_results

,HR_COST,DAY_COST,ppricdyp,pprichrp
ENS,,,,
gz1,390.0,1464.0,2172.093023,397.674419
gz10,383.0,1669.0,2812.121212,515.151515
gz11,415.0,1787.0,2590.000000,425.000000
gz16,0.0,0.0,1454.545455,372.727273
gz19,0.0,0.0,800.000000,200.000000
gz2,139.0,614.0,1141.176471,188.235294
gz3,129.0,812.0,857.142857,157.142857
gz4,223.0,915.0,905.263158,176.315789
gz5,246.0,1111.0,1584.615385,312.820513


In [33]:
df_parking_costs[df_parking_costs['HR_COST'] > 0]['ENS'].unique()

array(['gz1', 'gz2', 'gz3', 'gz4', 'gz5', 'gz6', 'gz7', 'gz8', 'gz9',
       'gz10', 'gz11', 'gz16', 'gz17', 'gz18', 'gz19', 'gz20', 'gz21',
       'gz22', 'gz24', 'gz26', 'gz27', 'gz28', 'gz29', 'gz30', 'gz31',
       'gz33', 'gz34', 'gz35', 'gz37', 'gz39', 'gz42', 'gz43', 'gz44'],
      dtype=object)